# Note on Data Import
In order to execute this notebook, you may need to change the lines 3-4 of the first cell and the `data_folder_path` variable in the second to match your own data path and your favourite import mathod.

If you wish to use Google Drive for data import, here a quick note on how it works:
1. althrough the file is publicly available on GD, it needs to be on your own Drive
1. still, you can transfer it from the public page to your own Drive directly, without having to download it locally and upload it
1. you may need to change the `data_folder_path` variable to match where in your GD you stored the dataset
1. after the dataset is available in your GD, the first cell will prompt you an URL. Click on it and it will return an autorization code, which you can copy and poste in the box below the cell
1. now you're good to go


In [2]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


<br><br/>
## 1
> _Normalize the `loan_lenders` table. In the normalized table, each row must have one `loan_id` and one `lender`_

In [3]:
data_folder_path = '/content/drive/' + 'My Drive/Colab Notebooks/' \
                    + 'additional-kiva-snapshot/'
lenders = pd.read_csv(data_folder_path + 'lenders.csv')
pd.set_option('display.max_rows', 10)
lenders

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
2349169,janet7309,Janet,NaN,NaN,NaN,1342097163,NaN,NaN,NaN,NaN,0
2349170,pj4198,NaN,NaN,NaN,NaN,1342097515,NaN,NaN,NaN,NaN,0
2349171,maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0
2349172,simone9846,Simone,NaN,NaN,NaN,1342100213,NaN,NaN,NaN,NaN,0


In [4]:
loan_lenders = pd.read_csv(data_folder_path + 'loans_lenders.csv')
loan_lenders

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."
...,...,...
1387427,678999,"michael43411218, carol5987, gooddogg1, chris41..."
1387428,1207353,"rjhoward1986, jeffrey6870, trolltech4460, elys..."
1387429,1206220,"vicky7746, gooddogg1, fairspirit, craig9729960..."
1387430,1206425,"rich6705, sergiiy9766, angela7509, barbara5610..."


In [5]:
lenders_list = loan_lenders.lenders.str.split(', ')
lengths = [len(l) for l in lenders_list.values]
lengths
norm_loan_lenders = pd.DataFrame({'loan_id': np.repeat(loan_lenders['loan_id'].values, lengths),
                       'lender': np.concatenate(lenders_list)})
norm_loan_lenders

,loan_id,lender
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


If a lender has contribuited more than once to a given loan, the row `(loan_id, lender_id)` will appear more than once in the table above. Indeed, this does happen:

In [6]:
# list all duplicate rows
# note: this code is relatively computationally expensive and for is here for
# display purposes only. You can avoid the execution of this cell without harm

norm_loan_lenders.loc[norm_loan_lenders.duplicated(keep = False), :]

,loan_id,lender
44,483738,danhostetler
45,483738,danhostetler
46,483738,danhostetler
120,563395,motoharu1020
121,563395,motoharu1020
...,...,...
28293698,1187825,hans8594
28293702,1187825,hans8594
28293703,1187825,hans8594
28293704,1187825,hans8594


Is this table normalized? If we consider the DataFrame index as a constituing part of the table, yes: the index acts as the primary key for this table. If the index is considered as an external entity then duplicate rows should be removed for the table to be considered normalized.

Which way is to be preferred? The answer comes down to what we consider atomic information here. If we think that each contribuition is a datum, then we should keep the table as it is. If the datum is the fact that an user has contribuited to a loan, no matter how many times he or she did, then we should remove duplicate lines.

<br><br/>
# 2
> _For each loan, add a column `duration` corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the `duration` must be missing._

In [7]:
loans = pd.read_csv(data_folder_path + 'loans.csv')
pd.set_option('display.max_columns', 100)
loans

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419602,988180,NaN,NaN,NaN,NaN,400.0,400.0,funded,Tailoring,Services,NaN,KE,Kenya,NaN,shared,0.1,KES,138.0,2015-12-01 04:49:50.000 +0000,2016-01-02 01:00:03.000 +0000,2015-11-23 08:00:00.000 +0000,2015-12-28 15:44:18.000 +0000,14.0,16,4,2,"#Parent, #Repeat Borrower, #Woman Owned Biz",NaN,NaN,monthly,field_partner
1419603,988213,Perlita,English,"Perlita is 52 years old, married and has three...","Perlita is 52 years old, married and has three...",300.0,300.0,funded,Pigs,Agriculture,to buy feeds and other supplies to raise her pigs,PH,Philippines,"Numancia, Aklan",shared,0.1,PHP,145.0,2015-12-01 05:13:30.000 +0000,2016-01-02 16:40:07.000 +0000,2015-11-24 08:00:00.000 +0000,2015-12-22 10:37:06.000 +0000,14.0,12,1,1,"#Animals, #Elderly, #Repeat Borrower, #Woman O...",female,true,irregular,field_partner
1419604,989109,Okyeso Nyame Group,English,Okyeso Nyame group will begin its third cycle ...,Okyeso Nyame group will begin its third cycle ...,2425.0,2425.0,funded,Bakery,Food,"to buy margarine, flour and sugar at wholesale...",GH,Ghana,Chorkor,shared,0.1,GHS,231.0,2015-12-02 10:56:28.000 +0000,2016-01-03 22:20:04.000 +0000,2015-11-13 08:00:00.000 +0000,2015-12-26 20:24:47.000 +0000,8.0,76,2,1,"user_favorite, #Parent, #Vegan, #Woman Owned B..

The two columns we're interesested in are coded as character string: it is therefore necessary to convert them to a time data format. If missing or invalid data are present, they're automatically trasformed to `<pandas._libs.tslibs.nattype.NaTType>` by Pandas. For example:

In [8]:
print('Missing data example:', loans['disburse_time'][64])
loans['disburse_time'] = pd.to_datetime(loans['disburse_time'])
loans['planned_expiration_time'] = pd.to_datetime(loans['planned_expiration_time'])
print('... it has been coded has NaT', loans['disburse_time'][64])

Missing data example: 2010-11-18 08:00:00.000 +0000
... it has been coded has NaT 2010-11-18 08:00:00+00:00


`NaT` values propagation rules are similar to those for the `nan` value of the `double` standard, in such a way that every algebraic operation on them returns `NaT`. Then if the difference below is computed, the result must be a `NaT` value:

In [9]:
print('NaT propagation example:', loans['disburse_time'][64], '-', \
      loans['planned_expiration_time'][64], "=", \
      (loans['disburse_time'] - loans['planned_expiration_time'])[64] )

NaT propagation example: 2010-11-18 08:00:00+00:00 - NaT = NaT


In [ ]:
loans['duration'] = loans['planned_expiration_time'] - loans['disburse_time']

In [11]:
loans[['planned_expiration_time', 'disburse_time', 'duration']]

,planned_expiration_time,disburse_time,duration
0,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,53 days 19:30:06
1,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,96 days 14:25:07
2,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,37 days 13:10:05
3,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,34 days 19:10:02
4,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,57 days 22:10:02
...,...,...,...
1419602,2016-01-02 01:00:03+00:00,2015-11-23 08:00:00+00:00,39 days 17:00:03
1419603,2016-01-02 16:40:07+00:00,2015-11-24 08:00:00+00:00,39 days 08:40:07
1419604,2016-01-03 22:20:04+00:00,2015-11-13 08:00:00+00:00,51 days 14:20:04
1419605,2016-01-05 08:50:02+00:00,2015-11-03 08:00:00+00:00,63 days 00:50:02


Please note that the `duration` column stores not only the number of days, but also the hours, ..., up to the nanoseconds. Althrough this isn't striclty necessary, having also the mumber of hours will allow us to have an higher accuracy in some computations.

If only the number of days is required, it can be easily extracted with: `loans['duration'].dt.days`.

<br><br/>
# 3
>_Find the lenders that have funded at least twice._

There are two ways in which the question can be interpreted:
1. find all the users who have funded at least twice, even if they did contribuite to one loan only
1. find all the users who have funded more than one loan

In order to solve the first one, we can simply count the occurences of each user id in the `norm_loan_lenders`.

In [12]:
norm_loan_lenders['lender'].value_counts()[norm_loan_lenders['lender'].value_counts() > 1]

gooddogg1           177645
trolltech4460       150762
gmct                128159
nms                 104314
themissionbeltco     81434
                     ...  
faye9556                 2
jane86872940             2
cheryl3850               2
andrew7860               2
suzanne64667456          2
Name: lender, Length: 901811, dtype: int64

For the second, we can simply do:

In [13]:
lenders[lenders['loan_purchase_num'] > 1]

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
6,rene7585,Rene,NaN,NaN,NaN,1461301636,NaN,NaN,2.0,NaN,0
7,harald2826,Harald,NaN,NaN,NaN,1461301670,NaN,NaN,2.0,NaN,0
11,jennifer4328,Jennifer,NaN,NaN,NaN,1461302712,NaN,NaN,3.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
2349158,rakhi,Rakhi,New York,New York,US,1342100607,Student,I care.,4.0,NaN,0
2349162,james75291930,James,NaN,NaN,NaN,1342096854,NaN,NaN,6.0,NaN,0
2349164,carol8279,Carol,NaN,NaN,NaN,1342099416,NaN,NaN,5.0,NaN,0
2349166,eric91401545,Eric,NaN,NaN,NaN,1342100719,NaN,NaN,2.0,NaN,0


And we get complete information on the lenders. We could get this amount of detail in the first case by using the result we already have and join it to the `lenders` table.

<br><br/>
# 4
> _For each country, compute how many loans have involved that country as borrowers._

The `country_code` variables can be used insted of `country_name`. If the database is consistent, the two ways will give equivalent results. If desired, the output can include both, using as Index a tuple containing the country name and the country code.

In [14]:
loans['country_name'].value_counts()

Philippines         285336
Kenya               143699
Peru                 86000
Cambodia             79701
El Salvador          64037
                     ...  
Papua New Guinea         1
Mauritania               1
Botswana                 1
Uruguay                  1
Canada                   1
Name: country_name, Length: 96, dtype: int64

<br><br/>
# 5
> _For each country, compute the overall amount of money borrowed._

The following code satisfies the request. Please note that all the amounts are expressed in USD.

In [15]:
loan_amount_by_ctry = loans.groupby(by = 'country_code').aggregate({'funded_amount': 'sum'})
loan_amount_by_ctry.rename({'funded_amount': 'received_amount'}, axis = 'columns', inplace = True)
loan_amount_by_ctry

,received_amount
country_code,
AF,1964650.0
AL,3992475.0
AM,20284675.0
AZ,14191350.0
BA,477250.0
...,...
XK,2862325.0
YE,3340025.0
ZA,1006525.0


<br><br/>
# 6
> _Like the previous point, but expressed as a percentage of the overall amount lent._

In [16]:
loan_amount_by_ctry['received_amount_(%)'] = 100 * loan_amount_by_ctry['received_amount']/sum(loan_amount_by_ctry['received_amount'])
loan_amount_by_ctry

,received_amount,received_amount_(%)
country_code,,
AF,1964650.0,0.173840
AL,3992475.0,0.353271
AM,20284675.0,1.794873
AZ,14191350.0,1.255710
BA,477250.0,0.042229
...,...,...
XK,2862325.0,0.253270
YE,3340025.0,0.295539
ZA,1006525.0,0.089062


<br><br/>
# 7
> _Like the three previous points, but split for each year (with respect to `disburse time`)._

In [18]:
loans_amount_by_yr_ctry = pd.DataFrame({'loan_count': loans.groupby(['country_code', loans.disburse_time.dt.year]).size()})
loans_amount_by_yr_ctry['received_amount'] = loans.groupby(['country_code', loans.disburse_time.dt.year]).aggregate({'funded_amount': 'sum'})
loans_amount_by_yr_ctry['received_amount_(%)'] = 100 * loans_amount_by_yr_ctry['received_amount']/sum(loans_amount_by_yr_ctry['received_amount'])
loans_amount_by_yr_ctry

loan_count  received_amount  received_amount_(%)
country_code disburse_time                                                  
AF           2007.0                408         194975.0             0.017283
             2008.0                370         365375.0             0.032387
             2009.0                678         581825.0             0.051573
             2010.0                632         563350.0             0.049936
             2011.0                247         245125.0             0.021728
...                                ...              ...                  ...
ZW           2013.0                426         678525.0             0.060145
             2014.0               2078        1307975.0             0.115940
             2015.0                600         723150.0             0.064100
             2016.0                808         787550.0             0.069809
             2017.0               1079        1214125.0             0.107621

[745 rows x 3 columns]

<br><br/>
# 8
> _For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount._

For this point it is necessary to remove the duplicates from the `norm_loan_lenders_table`.

In [ ]:
norm_loan_lenders2 = norm_loan_lenders.duplicated()
norm_loan_lenders2 = norm_loan_lenders.loc[~norm_loan_lenders2, :].copy()
norm_loan_lenders2.set_index('loan_id', inplace = True)
norm_loan_lenders2

,lender
loan_id,
483693,muc888
483693,sam4326
483693,camaran3922
483693,lachheb1865
483693,rebecca3499
...,...
1206425,trogdorfamily7622
1206425,danny6470
1206425,don6118


Now we can add a column containing the number of individual lenders that contribuited to each loan. (By doing so the table won't be normalized anymore, but it is convient to do so)

In [ ]:
norm_loan_lenders2['#_lenders'] = norm_loan_lenders2.groupby('loan_id').aggregate('count')
norm_loan_lenders2

,lender,#_lenders
loan_id,,
483693,muc888,40
483693,sam4326,40
483693,camaran3922,40
483693,lachheb1865,40
483693,rebecca3499,40
...,...,...
1206425,trogdorfamily7622,8
1206425,danny6470,8
1206425,don6118,8


Changing the index of the loans tables to the `loan_id` allows for quicker results when quering by this key. Exploiting this propriety will save time for the next points.

In [ ]:
assert loans['loan_id'].unique().size == loans.shape[0]
loans.set_index(loans['loan_id'], inplace = True)
loans.drop('loan_id', axis = 'columns', inplace = True)
loans

,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration
loan_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner,53 days 19:30:06
657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner,96 days 14:25:07
658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37 days 13:10:05
659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,34 days 19:10:02
656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,57 days 22:10:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988180,NaN,NaN,NaN,NaN,400.0,400.0,funded,Tailoring,Services,NaN,KE,Kenya,NaN,shared,0.1,KES,138.0,2015-12-01 04:49:50.000 +0000,2016-01-02 01:00:03+00:00,2015-11-23 08:00:00+00:00,2015-12-28 15:44:18.000 +0000,14.0,16,4,2,"#Parent, #Repeat Borrower, #Woman Owned Biz",NaN,NaN,monthly,field_partner,39 days 17:00:03
988213,Perlita,English,"Perlita is 52 years old, married and has three...","Perlita is 52 years old, married and has three...",300.0,300.0,funded,Pigs,Agriculture,to buy feeds and other supplies to raise her pigs,PH,Philippines,"Numancia, Aklan",shared,0.1,PHP,145.0,2015-12-01 05:13:30.000 +0000,2016-01-02 16:40:07+00:00,2015-11-24 08:00:00+00:00,2015-12-22 10:37:06.000 +0000,14.0,12,1,1,"#Animals, #Elderly, #Repeat Borrower, #Woman O...",female,true,irregular,field_partner,39 days 08:40:07
989109,Okyeso Nyame Group,English,Okyeso Nyame group will begin its third cycle ...,Okyeso Nyame group will begin its third cycle ...,2425.0,2425.0,funded,Bakery,Food,"to buy margarine, flour and sugar at wholesale...",GH,Ghana,Chorkor,shared,0.1,GHS,231.0,2015-12-02 10:56:28.000 +0000,2016-01-03 22:20:04+00:00,2015-11-13 08:00:00+00:00,2015-12-26 20:24:47.000 +0000

In [ ]:
# computes the amount funded by each lender, for each loan
norm_loan_lenders2['total_fa'] = loans.loc[norm_loan_lenders2.index, 'funded_amount'].copy()
norm_loan_lenders2['per_user_fa'] = norm_loan_lenders2['total_fa']/norm_loan_lenders2['#_lenders']
norm_loan_lenders2

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:961: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


,lender,#_lenders,total_fa,per_user_fa
loan_id,,,,
483693,muc888,40,1225.0,30.625
483693,sam4326,40,1225.0,30.625
483693,camaran3922,40,1225.0,30.625
483693,lachheb1865,40,1225.0,30.625
483693,rebecca3499,40,1225.0,30.625
...,...,...,...,...
1206425,trogdorfamily7622,8,325.0,40.625
1206425,danny6470,8,325.0,40.625
1206425,don6118,8,325.0,40.625


In [ ]:
# refactor dataframe in order to keep the relevant information only
lender_fa = norm_loan_lenders2.groupby('lender').aggregate({'per_user_fa': 'sum'})
lender_fa

,per_user_fa
lender,
000,1739.571571
00000,1422.820392
0002,2440.510891
00mike00,52.631579
0101craign0101,2701.900177
...,...
zzmcfate,64294.370209
zzpaghetti9994,62.500000
zzrvmf8538,553.059450


<br><br/>
# 9
> _For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known._

In [ ]:
# add per_user_fa (funded amount) column to lenders table
lenders = lenders.join(other = lender_fa, on = 'permanent_name')
lenders.loc[lenders['per_user_fa'].isna(), 'per_user_fa'] = 0
lenders

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited,per_user_fa
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0,47.383721
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0,419.568943
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0,0.000000
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0,159.671338
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2349169,janet7309,Janet,NaN,NaN,NaN,1342097163,NaN,NaN,NaN,NaN,0,0.000000
2349170,pj4198,NaN,NaN,NaN,NaN,1342097515,NaN,NaN,NaN,NaN,0,0.000000
2349171,maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0,110.507246
2349172,simone9846,Simone,NaN,NaN,NaN,1342100213,NaN,NaN,NaN,NaN,0,0.000000


In [ ]:
# add 'received_amount' column to 'loan_amount_by_country' table
temp = lenders.groupby('country_code').aggregate({'per_user_fa': 'sum'})
loan_amount_by_ctry = loan_amount_by_ctry.join(other = temp)
loan_amount_by_ctry.rename({'per_user_fa': 'given_amount'}, axis = 'columns', inplace = True)
loan_amount_by_ctry

,received_amount,received_amount_(%),given_amount
country_code,,,
AF,1964650.0,0.001738,104374.931038
AL,3992475.0,0.003533,7998.358521
AM,20284675.0,0.017949,22707.965557
AZ,14191350.0,0.012557,8785.629857
BA,477250.0,0.000422,62127.194187
...,...,...,...
XK,2862325.0,0.002533,10775.974872
YE,3340025.0,0.002955,16393.414355
ZA,1006525.0,0.000891,526849.076548


In [ ]:
# assign money from lenders without country as specified in the text
na_sum = lenders.loc[lenders['country_code'].isna(), 'per_user_fa'].sum()
loan_amount_by_ctry['given_amount'] += na_sum*loan_amount_by_ctry['received_amount_(%)'] / 100
loan_amount_by_ctry

,received_amount,received_amount_(%),given_amount
country_code,,,
AF,1964650.0,0.001738,4.909072e+05
AL,3992475.0,0.003533,7.934922e+05
AM,20284675.0,0.017949,4.013588e+06
AZ,14191350.0,0.012557,2.800843e+06
BA,477250.0,0.000422,1.560231e+05
...,...,...,...
XK,2862325.0,0.002533,5.739201e+05
YE,3340025.0,0.002955,6.735219e+05
ZA,1006525.0,0.000891,7.248764e+05


In [ ]:
# finally, compute the difference
loan_amount_by_ctry['diff'] = loan_amount_by_ctry['given_amount'] - loan_amount_by_ctry['received_amount']
loan_amount_by_ctry

,received_amount,received_amount_(%),given_amount,diff
country_code,,,,
AF,1964650.0,0.001738,4.909072e+05,-1.473743e+06
AL,3992475.0,0.003533,7.934922e+05,-3.198983e+06
AM,20284675.0,0.017949,4.013588e+06,-1.627109e+07
AZ,14191350.0,0.012557,2.800843e+06,-1.139051e+07
BA,477250.0,0.000422,1.560231e+05,-3.212269e+05
...,...,...,...,...
XK,2862325.0,0.002533,5.739201e+05,-2.288405e+06
YE,3340025.0,0.002955,6.735219e+05,-2.666503e+06
ZA,1006525.0,0.000891,7.248764e+05,-2.816486e+05


<br><br/>
# 10
> _Which country has the highest ratio between the difference computed at the previous point and the population?_

In [ ]:
country_stats = pd.read_csv(data_folder_path + 'country_stats.csv')
country_stats

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Somalia,SO,SOM,Africa,Eastern Africa,14742523,NaN,NaN,NaN,NaN,NaN,NaN,Somalia
170,Central African Republic,CF,CAF,Africa,Middle Africa,4659080,NaN,0.352440,51.458,7.098980,4.230000,587.473961,Central African Republic
171,Samoa,WS,WSM,Oceania,Polynesia,196440,NaN,0.702000,NaN,NaN,NaN,NaN,Samoa
172,Palestine,PS,PS,Asia,Western Asia,4920724,NaN,0.677000,NaN,NaN,NaN,NaN,Palestine


In [ ]:
population = pd.Series(data = country_stats['population'].values, index = country_stats['country_code'])
population

country_code
IN    1339180127
NG     190886311
MX     129163276
PK     197015955
BD     164669751
         ...    
SO      14742523
CF       4659080
WS        196440
PS       4920724
XK       1895250
Length: 174, dtype: int64

In [ ]:
loan_amount_by_ctry['diff_per_cap'] = (loan_amount_by_ctry['diff']/population)
loan_amount_by_ctry

,received_amount,received_amount_(%),given_amount,diff,diff_per_cap
country_code,,,,,
AF,1964650.0,0.001738,4.909072e+05,-1.473743e+06,-0.041479
AL,3992475.0,0.003533,7.934922e+05,-3.198983e+06,-1.091733
AM,20284675.0,0.017949,4.013588e+06,-1.627109e+07,-5.552419
AZ,14191350.0,0.012557,2.800843e+06,-1.139051e+07,-1.159034
BA,477250.0,0.000422,1.560231e+05,-3.212269e+05,-0.091595
...,...,...,...,...,...
XK,2862325.0,0.002533,5.739201e+05,-2.288405e+06,-1.207442
YE,3340025.0,0.002955,6.735219e+05,-2.666503e+06,-0.094388
ZA,1006525.0,0.000891,7.248764e+05,-2.816486e+05,-0.004966


In [ ]:
# the following method is able to find all maxima, non only the first one
mask = loan_amount_by_ctry['diff_per_cap'] == loan_amount_by_ctry['diff_per_cap'].max()
loan_amount_by_ctry.loc[mask, :]

,received_amount,received_amount_(%),given_amount,diff,diff_per_cap
country_code,,,,,
CA,50000.0,0.000044,7.691198e+07,7.686198e+07,2.098666


From this result we can see that Canada has the highest surplus: in this country, on average, each individual has lended 2$ more than he or she has borrowed.

<br><br/>
# 11
> _Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?_ 

In [ ]:
population_nb = pd.Series(data = country_stats['population_below_poverty_line'].values, index = country_stats['country_code'])
population_nb = population - population_nb
population_nb

country_code
IN    1.339180e+09
NG    1.908862e+08
MX    1.291632e+08
PK    1.970159e+08
BD    1.646697e+08
          ...     
SO             NaN
CF             NaN
WS             NaN
PS             NaN
XK    1.895220e+06
Length: 174, dtype: float64

In [ ]:
loan_amount_by_ctry['diff_nb'] = (loan_amount_by_ctry['diff']/population_nb)
loan_amount_by_ctry

,received_amount,received_amount_(%),given_amount,diff,diff_per_cap,diff_nb
country_code,,,,,,
AF,1964650.0,0.001738,4.909072e+05,-1.473743e+06,-0.041479,-0.041479
AL,3992475.0,0.003533,7.934922e+05,-3.198983e+06,-1.091733,-1.091739
AM,20284675.0,0.017949,4.013588e+06,-1.627109e+07,-5.552419,-5.552480
AZ,14191350.0,0.012557,2.800843e+06,-1.139051e+07,-1.159034,-1.159034
BA,477250.0,0.000422,1.560231e+05,-3.212269e+05,-0.091595,-0.091596
...,...,...,...,...,...,...
XK,2862325.0,0.002533,5.739201e+05,-2.288405e+06,-1.207442,-1.207461
YE,3340025.0,0.002955,6.735219e+05,-2.666503e+06,-0.094388,-0.094388
ZA,1006525.0,0.000891,7.248764e+05,-2.816486e+05,-0.004966,-0.004966


In [ ]:
mask = loan_amount_by_ctry['diff_nb'] == loan_amount_by_ctry['diff_nb'].max()
loan_amount_by_ctry.loc[mask, :]

,received_amount,received_amount_(%),given_amount,diff,diff_per_cap,diff_nb
country_code,,,,,,
CA,50000.0,0.000044,7.691198e+07,7.686198e+07,2.098666,2.098667


<br><br/>
# 12
> _For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year._

This point can be completed by defining a function and iterating over the rows of the `loans` dataframe by using `apply`. However, there is an issue: the function has to return a variable-lentgh `pd.Series`, which is an operation that Pandas doesn't support. A possibile workaround consists in returning `None` and appending the result to a previously defined list.

In [ ]:
def calc_12(x, rl):
  start = x['disburse_time']
  end = x['planned_expiration_time']
  if type(start) == pd._libs.tslibs.nattype.NaTType or type(end) == pd._libs.tslibs.nattype.NaTType:
    return None
  if end < start:     # see note1 at the bottom
    return None
  years = np.arange(start.year, end.year+1, dtype = 'int64')
  years = pd.to_datetime(years, format = '%Y')
  years = years.tz_localize(start.tz).tz_convert(start.tz)
  weights = np.empty(shape = years.size, dtype = 'float64')
  tot = x['funded_amount']
  time_unit = pd.to_timedelta(1, unit='D')
  if years.size == 1:
    weights = 1
  else:
    weights[0] = (years[1] - start)/time_unit
    weights[-1] = (end - years[-1])/time_unit
    if years.size > 2:
      weights[1:-1] = 365
    weights = weights/(x['duration']/time_unit)
  result = pd.Series(data = weights*tot, index = years)
  rl.append(result)
  return None

In [ ]:
rl = list()
loans.apply(calc_12, axis = 'columns', rl = rl)

loan_id
657307    None
657259    None
658010    None
659347    None
656933    None
          ... 
988180    None
988213    None
989109    None
989143    None
989240    None
Length: 1419607, dtype: object

In [ ]:
loans_per_year = pd.concat(rl, axis = 'rows')
loans_per_year.index.set_names('year', inplace = True)
loans_per_year.rename('amount', inplace = True)
loans_per_year

2013-01-01 00:00:00+00:00     22.454481
2014-01-01 00:00:00+00:00    102.545519
2013-01-01 00:00:00+00:00     48.308791
2014-01-01 00:00:00+00:00    351.691209
2014-01-01 00:00:00+00:00    400.000000
                                ...    
2016-01-01 00:00:00+00:00    137.734219
2015-01-01 00:00:00+00:00     93.070364
2016-01-01 00:00:00+00:00      6.929636
2015-01-01 00:00:00+00:00    166.843283
2016-01-01 00:00:00+00:00      8.156717
Length: 1181619, dtype: float64

In [ ]:
loans_per_year = loans_per_year.groupby('year').aggregate('sum')
loans_per_year

year
2011-01-01 00:00:00+00:00    6.450017e+05
2012-01-01 00:00:00+00:00    1.086246e+08
2013-01-01 00:00:00+00:00    1.202665e+08
2014-01-01 00:00:00+00:00    1.454674e+08
2015-01-01 00:00:00+00:00    1.478736e+08
2016-01-01 00:00:00+00:00    1.448194e+08
2017-01-01 00:00:00+00:00    1.561052e+08
2018-01-01 00:00:00+00:00    3.303027e+06
Name: amount, dtype: float64

<br><br/>
# Notes
## Note 1
If loans where formatted correctly, this check would not be necessary. However, at index 1190028 the `planned_expiration_time` is before the `disburse_time`. A glance at the `loans` table makes me think that the two have been flipped, for some unkown reason. I could fix this issue, but given that the reason is not 100% clear, I prefer to leave it alone.

In [ ]:
loans.loc[[1190028, 1190028],  ['planned_expiration_time', 'disburse_time']]

,planned_expiration_time,disburse_time
loan_id,,
1190028,2016-12-27 15:00:04+00:00,2017-02-01 08:00:00+00:00
1190028,2016-12-27 15:00:04+00:00,2017-02-01 08:00:00+00:00
